In [81]:
import re
import csv
import pandas as pd
import numpy as np

In [82]:
import re
import csv

# 🔧 사용자 정의: 추적할 이벤트 목록
# tracked_events = ["prepare", "Igather", "solve_reduced", "Iscatter", "solve_remain", "solve_x"]
tracked_events = ["solve_z", "solve_y", "solve_x"]

# 🔧 파일 설정
input_file = "time_log.txt"
output_file = "time_parsed.csv"

# 정규식: [이벤트] elapsed: ... ns
pattern_event = re.compile(r"\[(.+?)\]\s*elapsed:\s*(\d+)\s*ms")

# 결과 리스트
records = []
current_rank = 0
event_counter = 0
events_per_rank = 0

with open(input_file, "r") as f:
    for line in f:
        line = line.strip()
        m = pattern_event.search(line)
        if m:
            label = m.group(1)
            elapsed = int(m.group(2))

            if label in tracked_events:
                records.append((current_rank, label, elapsed))
                event_counter += 1

                # 랭크 판단: 이벤트 수를 기준으로 랭크 추정
                if events_per_rank == 0 and records:
                    # 처음 들어온 이벤트로부터 한 랭크의 이벤트 개수 추정
                    events_per_rank = tracked_events.count(records[0][1])

                if events_per_rank > 0 and event_counter % events_per_rank == 0:
                    current_rank += 1

# CSV 저장
with open(output_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Rank", "Event", "Time_ns"])
    for r in records:
        writer.writerow(r)

print(f"[✓] CSV 파일 저장 완료: {output_file}")


[✓] CSV 파일 저장 완료: time_parsed.csv


In [83]:
data = pd.read_csv("time_parsed.csv")

ddata = np.zeros(len(tracked_events))

for i in range(len(tracked_events)):
    event_name = tracked_events[i]
    time = np.max(data["Time_ns"][data["Event"]==event_name])
    ddata[i] = time

#ddata
pd.DataFrame(ddata)

,0
0,12.0
1,13.0
2,32.0
